In [1]:
import pandas as pd
import re
import math
import openpyxl

In [2]:
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# AOPA

## Load output - from AOPAdata - get cleancali

### Data prep - clinic name ready in cleancali.xlsx

In [3]:
frs = pd.read_excel('output.xlsx', sheet_name = 'CALIFORNIA')

In [4]:
frs.head(2)

,Clinic_name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,Molina Orthopedic Labs Inc./dba Crown City Ort...,"610 N Santa Anita Ave, Ste 100","Arcadia , CA 91006",(626) 431-2890,Fax (626) 431-2892,mailto:ryan@crowncityortho.com,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sierra Prosthetics -Orthotics ❹,3227 Professional Dr,"Auburn , CA 95602",(530) 885-9922,mailto:sierrapo@gv.net,NaN,NaN,"BAKERSFIELD TUCSON Alicia Lenhart, RFOM",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# [x.lower() for x in dataframe['Provider Organization Name (Legal Business Name)']]

In [6]:
frs['Clinic_name'] = frs['Clinic_name'].str.replace(' C ', '').str.replace('❹', '').str.replace(' S ', '')

In [7]:
frs.to_excel('cleancali.xlsx',index=False)

##### Attempt for cleaning pipeline, havn't achieve so ...

In [8]:
# state = 'CA'

In [9]:
# frame = pd.read_excel('cleancali.xlsx')

In [10]:
# frame.columns

In [11]:
# frame.head(2)

In [12]:
# keepf = frame[(frame['Address'].str.contains(state)) | 
#           (frame['County'].str.contains(state)) | 
#           (frame['Unnamed: 3'].str.contains(state))]

In [13]:
# keepf

In [14]:
# def starts_with_number(s):
#     return re.match(r'^\d', s) is not None

In [15]:
# column_to_check = 'Address'
# new_data = []

# for index, row in keepf.iterrows():
#     value = row[column_to_check]
#     # print(value)
#     new_row = [None] + row.tolist() if not starts_with_number(value) else row.tolist()
#     new_data.append(new_row)

# new_df = pd.DataFrame(new_data, columns=[''] + df.columns.tolist())

##### ... manual cleaning for formatted column 

## Load cleancali2 - city, state, zip columns in cleancali3.xlsx

In [16]:
frame2 = pd.read_excel('cleancali2.xlsx')

In [17]:
frame2.head(2)

,Clinic_name,Address,City，State，Zipcode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Molina Orthopedic Labs Inc./dba Crown City Ort...,"610 N Santa Anita Ave, Ste 100","Arcadia , CA 91006",(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,Sierra Prosthetics -Orthotics,3227 Professional Dr,"Auburn , CA 95602",(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
frame2['City，State，Zipcode']

0                             Arcadia ,  CA 91006
1                             Auburn  ,  CA 95602
2                          Aliso Viejo, CA 92656 
3                         Arroyo Grande, CA 93420
4                      Bakersfield, CA 93301-1816
                          ...                    
170                     Ventura  ,  CA 93003-7710
171                            Vista  ,  CA 92081
172                       Vista  ,  CA 92081-6056
173    Whittier Medical Plz Whittier  ,  CA 90601
174                       Whittier, CA 90606-2583
Name: City，State，Zipcode, Length: 175, dtype: object

In [19]:
type(frame2['City，State，Zipcode'][37])

float

In [20]:
formatted_values = []

for value in frame2['City，State，Zipcode']:
    if isinstance(value, float) and math.isnan(value):
        city = ""
        state = ""
        zip_code = ""

    # Convert the value to string if it's not already
    if not isinstance(value, str):
        value = str(value)
        city = ""
        state = ""
        zip_code = ""
        
    if value.strip():
        parts = re.split(", |,\xa0", value)
        city = parts[0].strip()
        
        if len(parts) == 1:  # If only city name is present
            state = "CA"
            zip_code = ""
        else:
            state_zip = parts[1].split("A ")
            state = state_zip[0].strip() + 'A'
            zip_code = state_zip[1].strip()
            
        formatted_row = {
            'City': city,
            'State': state,
            'ZipCode': zip_code
        }
        
        formatted_values.append(formatted_row)

In [21]:
formatted_df = pd.DataFrame(formatted_values)

In [22]:
formatted_df.head(2)

,City,State,ZipCode
0,Arcadia,CA,91006
1,Auburn,CA,95602


In [23]:
temp_df3 = pd.concat([frame2, formatted_df], axis=1)

In [24]:
temp_df3.columns

Index(['Clinic_name', 'Address', 'City，State，Zipcode', 'Phone', 'FAX', 'Email',
       'Website', 'Name', 'Other_info', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'City', 'State', 'ZipCode'],
      dtype='object')

In [25]:
column_order = ['Clinic_name', 'Address', 'City', 'State', 'ZipCode', 'Phone', 'FAX', 'Email',
       'Website', 'Name', 'Other_info', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'City，State，Zipcode']
temp_df3 = temp_df3[column_order]

In [26]:
temp_df3 = temp_df3.drop(columns=['City，State，Zipcode'])

In [27]:
temp_df3['City'] = ["" if cell=='nan' else cell for cell in temp_df3['City']]

In [28]:
temp_df3['ZipCode'] = [cell[:5] for cell in temp_df3['ZipCode']]

In [29]:
temp_df3.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Molina Orthopedic Labs Inc./dba Crown City Ort...,"610 N Santa Anita Ave, Ste 100",Arcadia,CA,91006,(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,Sierra Prosthetics -Orthotics,3227 Professional Dr,Auburn,CA,95602,(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
temp_df3.to_excel('cleancali3.xlsx',index=False)

## Load result_df  - use zip code to filter clinics by page

In [31]:
result_df = pd.read_excel('cleancali3.xlsx')

In [32]:
# result_df['ZipCode'] = pd.to_numeric(result_df['ZipCode'], errors='coerce', downcast='integer')

In [33]:
area1 = [92074,92131,92199,92128,92064,92129,92198,92071,92126,92127,92145,92072,92130,92124,92121,92119,92014,92067,92025,92091,92120,92029,92123,92040,92122,91942,92020,92022,92111,92182,92030,92033,92046,92021,92117,91943,91944,92075,92037,92115,91941,92108,92116,92007,92027,92024,92096,91946,91976,91979,92078,92093,92065,91945,92092,92105,92038,92039,92104,92079,92019,92109,92110,92009,92023,92103,92169,91977,92102,92013,92134,92147,92114,92140,92112,92137,92138,92142,92143,92149,92150,92158,92159,92160,92161,92163,92165,92166,92167,92168,92170,92171,92172,92174,92175,92176,92177,92179,92186,92187,92191,92192,92193,92195,92196,92197,92132,92101,92069,92113,91978,91903,92011,92107,92139,91902,91951,92081,92136,92026,92135,91950,92010,92106,92178,92155,91908,92118,92084,92152,91914,91921,92083,92085,92008,91909,91912,91910,92082,91913,91901,92056,92018,91915,91935,91911,91916,92070,92054,92003,92057,91933,92049,92051,92052,92153,92061,91932,92154,92173,92060]

In [34]:
area2 = [92607,92652,92654,92651,92657,92656,92677,92629,92625,92603,92637,92653,92609,92662,92661,92693,92617,92697,92660,92690,92624,92694,92691,92612,92658,92659,92616,92619,92623,92698,92692,92663,92628,92650,92630,92627,92673,92614,92604,92618,92606,92626,92707,92674,92688,92620,92646,92782,92675,92780,92615,92610,92781,92704,92705,92678,92708,92728,92701,92702,92711,92712,92602,92799,92679,92735,92648,92605,92703,92843,92706,92672,92866,92647,92856,92857,92859,92863,92864,92655,92868,92685,92844,92869,92676,92683,92840,92842,92846,92867,92649,92684,92861,90742,92862,92802,92841,92865,92806,90743,92805,90680,92845,92804,92803,92812,92814,92815,92816,92817,92825,92850,92899,92807,92808,90740,92811,92882,92809,92883,92530,92801,90720,90630,92055,90721,92870,92831,92832,90620,90803,92885,92871,92834,92836,92837,92838,92887,92886,90840,90815,90814,90622,90624,92833,90716,90623,90621,92835,90804,90715,90831,90833,90808,92877,92878,92823]

In [35]:
area3 = [92359,92346,92374,92375,92399,92382,92554,92373,92369,92385,92320,92318,92556,92408,92354,92350,92357,92341,92324,92321,92401,92410,92404,92402,92406,92413,92415,92418,92423,92427,92378,92317,92403,92557,92405,92391,92305,92352,92313,92411,92555,92223,92315,92339,92553,92325,92333,92507,92407,92314,92322,92521,92551,92376,92316,92377,92501,92518,92386,92506,92220,92571,92508,92334,92502,92513,92514,92516,92517,92519,92522,92567,92331,92552,92582,92335,92336,92509,92337,92504,92230,92583,92308,92572,92599,92581,92548,92503,92345,91739,92546,92585,92505,92545,92340,92570,91752,92344,92282,91730,92586,92543,91737]

In [36]:
zips_ca = pd.read_csv("zip-code-list.csv")

In [37]:
92074 in zips_ca["zip_code"].tolist()

True

In [38]:
len(area1) + len(area2) + len(area3)

421

In [39]:
len(zips_ca["zip_code"])

2664

In [40]:
leftset = set(zips_ca["zip_code"]) - set(area1) - set(area2) - set(area3)
area4 = list(leftset)

In [41]:
filtered1 = result_df[result_df['ZipCode'].isin(area1)]
filtered1.shape
# filtered1

(17, 14)

In [42]:
filtered2 = result_df[result_df['ZipCode'].isin(area2)]
filtered2.shape
# filtered2

(17, 14)

In [43]:
filtered3 = result_df[result_df['ZipCode'].isin(area3)]
filtered3.shape
# filtered3

(6, 14)

In [44]:
filtered4 = result_df[result_df['ZipCode'].isin(area4)]
filtered4.shape
# filtered4

(123, 14)

In [45]:
workbook = openpyxl.load_workbook('cleancali3.xlsx')

In [46]:
page_names = ['area1', 'area2', 'area3', 'area4']
dataframes = [filtered1, filtered2, filtered3, filtered4]

In [47]:
for df, page_name in zip(dataframes, page_names):
    new_worksheet = workbook.create_sheet(title=page_name)

    # Write DataFrame to the new worksheet
    for row in dataframe_to_rows(df, index=False, header=True):
        new_worksheet.append(row)

In [48]:
workbook.save('cleancali4.xlsx')

## Update cleancali3 - clean rows/cols to combine multiple df

In [49]:
frame3 = pd.read_excel('cleancali3.xlsx')

In [50]:
frame3.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Molina Orthopedic Labs Inc./dba Crown City Ort...,"610 N Santa Anita Ave, Ste 100",Arcadia,CA,91006.0,(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,Sierra Prosthetics -Orthotics,3227 Professional Dr,Auburn,CA,95602.0,(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
frame3["ZipCode"] = frame3["ZipCode"].apply(lambda x: str(int(x)) if not pd.isna(x) else "")

In [52]:
frame3.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Molina Orthopedic Labs Inc./dba Crown City Ort...,"610 N Santa Anita Ave, Ste 100",Arcadia,CA,91006,(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,Sierra Prosthetics -Orthotics,3227 Professional Dr,Auburn,CA,95602,(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
[x.lower().strip() for x in frame3["Clinic_name"]][:15]

['molina orthopedic labs inc./dba crown city orthopedic',
 'sierra prosthetics -orthotics',
 'medequip inc.',
 'north county prosthetics &amp; orthotics',
 'achilles prosthetics &amp; orthotics inc.',
 'hanger clinic',
 'hanger clinic',
 'human designs prosthetic &amp; orthotic laboratory inc.',
 'north county prosthetics &amp; orthotics',
 'hanger clinic',
 'townsend design (thuasne usa)   +',
 'valley institute of prosthetics &amp; orthotics',
 'supply inc.',
 'hanger clinic',
 'prosthetic &amp; orthotic group los angeles']

In [54]:
def remove_amp(s):
    return s.replace("&amp;", "&").replace("+","")

In [55]:
def extract_y(s):
    match = re.search(r'/dba\s+(.+)', s)
    if match:
        return match.group(1)
    return s

In [56]:
frame3["Clinic_name"] = frame3["Clinic_name"].apply(lambda x: extract_y(x))
frame3["Clinic_name"] = frame3["Clinic_name"].apply(lambda x: remove_amp(x))

In [57]:
[x.lower().strip() for x in frame3["Clinic_name"]][:15]

['crown city orthopedic',
 'sierra prosthetics -orthotics',
 'medequip inc.',
 'north county prosthetics & orthotics',
 'achilles prosthetics & orthotics inc.',
 'hanger clinic',
 'hanger clinic',
 'human designs prosthetic & orthotic laboratory inc.',
 'north county prosthetics & orthotics',
 'hanger clinic',
 'townsend design (thuasne usa)',
 'valley institute of prosthetics & orthotics',
 'supply inc.',
 'hanger clinic',
 'prosthetic & orthotic group los angeles']

In [58]:
cols_tolower = ['Clinic_name', 'Address', 'City', 'State', 'ZipCode']

In [59]:
frame3["Clinic_name"][:5].tolist()

['Crown City Orthopedic ',
 'Sierra Prosthetics -Orthotics   ',
 'Medequip Inc. ',
 'North County Prosthetics & Orthotics   ',
 'Achilles Prosthetics & Orthotics Inc.']

In [60]:
for col in cols_tolower:
    frame3[col] = frame3[col].apply(lambda x: x.lower().strip() if isinstance(x, str) else x)

In [61]:
frame3["Clinic_name"][:5].tolist()

['crown city orthopedic',
 'sierra prosthetics -orthotics',
 'medequip inc.',
 'north county prosthetics & orthotics',
 'achilles prosthetics & orthotics inc.']

In [62]:
frame3.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,crown city orthopedic,"610 n santa anita ave, ste 100",arcadia,ca,91006,(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,sierra prosthetics -orthotics,3227 professional dr,auburn,ca,95602,(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN


# NPI

In [63]:
dataframe = pd.read_excel('list_caliclinic.xlsx')

In [64]:
dataframe.head(2)

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,City,State2,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number,Zip5,Address,taxonomy
0,1548468614,2,DIMENSION PROSTHETICS & ORTHOTICS,33374 DOWE AVE,UNION CITY,CA,94587,5103243400,94587,33374 DOWE AVE,335E00000X
1,1386652774,2,"HANGER PROSTHETICS & ORTHOTICS WEST, INC.",3460 ROBIN LN,CAMERON PARK,CA,95682,5306764856,95682,3460 ROBIN LN,335E00000X


In [65]:
dataframe.columns

Index(['NPI', 'Entity Type Code',
       'Provider Organization Name (Legal Business Name)',
       'Provider First Line Business Practice Location Address', 'City',
       'State2', 'Provider Business Practice Location Address Postal Code',
       'Provider Business Practice Location Address Telephone Number', 'Zip5',
       'Address', 'taxonomy'],
      dtype='object')

In [66]:
column_order = ['Provider Organization Name (Legal Business Name)','Address','City',
                'State2','Zip5', 'Provider Business Practice Location Address Telephone Number',
                'NPI', 'taxonomy', 'Entity Type Code', 'Provider First Line Business Practice Location Address',
               'Provider Business Practice Location Address Postal Code']
dataframe = dataframe[column_order]

In [67]:
dataframe.rename(columns={'Provider Organization Name (Legal Business Name)': 'Clinic_name', 
                            'State2': 'State', 'Zip5': 'ZipCode',
                           'Provider Business Practice Location Address Telephone Number': 'Phone'}, inplace=True)

In [68]:
dataframe.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,NPI,taxonomy,Entity Type Code,Provider First Line Business Practice Location Address,Provider Business Practice Location Address Postal Code
0,DIMENSION PROSTHETICS & ORTHOTICS,33374 DOWE AVE,UNION CITY,CA,94587,5103243400,1548468614,335E00000X,2,33374 DOWE AVE,94587
1,"HANGER PROSTHETICS & ORTHOTICS WEST, INC.",3460 ROBIN LN,CAMERON PARK,CA,95682,5306764856,1386652774,335E00000X,2,3460 ROBIN LN,95682


In [69]:
for col in cols_tolower:
    dataframe[col] = dataframe[col].apply(lambda x: x.lower().strip() if isinstance(x, str) else x)

In [70]:
dataframe.head(2)

,Clinic_name,Address,City,State,ZipCode,Phone,NPI,taxonomy,Entity Type Code,Provider First Line Business Practice Location Address,Provider Business Practice Location Address Postal Code
0,dimension prosthetics & orthotics,33374 dowe ave,union city,ca,94587,5103243400,1548468614,335E00000X,2,33374 DOWE AVE,94587
1,"hanger prosthetics & orthotics west, inc.",3460 robin ln,cameron park,ca,95682,5306764856,1386652774,335E00000X,2,3460 ROBIN LN,95682


In [71]:
frame3.head(3)

,Clinic_name,Address,City,State,ZipCode,Phone,FAX,Email,Website,Name,Other_info,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,crown city orthopedic,"610 n santa anita ave, ste 100",arcadia,ca,91006,(626) 431-2890,Fax (626) 431-2892,ryan@crowncityortho.com,NaN,NaN,NaN,NaN,NaN,NaN
1,sierra prosthetics -orthotics,3227 professional dr,auburn,ca,95602,(530) 885-9922,NaN,sierrapo@gv.net,NaN,NaN,NaN,NaN,NaN,NaN
2,medequip inc.,27 brookline,aliso viejo,ca,92656,(949) 443-4414,NaN,soren@medequiportho.com,http://www.medequiportho.com/,"Soren Lind, President","Kasey Bancroft, O "" ce Manager","Brian Keim, Area Manager Trish Terena, Sales M...",NaN,NaN


# AOPA-NPI

In [72]:
dataframe['combineid'] = dataframe['Clinic_name'].str.split().str[0] + dataframe['ZipCode'].astype(str)
frame3['combineid'] = frame3['Clinic_name'].str.split().str[0] + frame3['ZipCode'].astype(str)

In [73]:
dataframe['combineid2'] = dataframe['Clinic_name'].str.split().str[0] + dataframe['Address'].str.split().str[0]
frame3['combineid2'] = frame3['Clinic_name'].str.split().str[0] + frame3['Address'].str.split().str[0]

In [74]:
dataframe['combineid3'] = dataframe['ZipCode'].astype(str) + dataframe['Address'].str.split().str[0]
frame3['combineid3'] = frame3['ZipCode'].astype(str) + frame3['Address'].str.split().str[0]

In [75]:
merged1 = pd.merge(dataframe, frame3, on=['combineid'], how='inner')
merged2 = pd.merge(dataframe, frame3, on=['combineid2'], how='inner')
merged3 = pd.merge(dataframe, frame3, on=['combineid3'], how='inner')

In [127]:
merged1 = merged1.fillna('NaN')
merged2 = merged2.fillna('NaN')
merged3 = merged3.fillna('NaN')

In [129]:
merged_df = pd.concat([merged1, merged2, merged3])

In [132]:
merged_df.shape

(292, 34)

In [139]:
merged_df.columns

Index(['Clinic_name_x', 'Address_x', 'City_x', 'State_x', 'ZipCode_x',
       'Phone_x', 'NPI', 'taxonomy', 'Entity Type Code',
       'Provider First Line Business Practice Location Address',
       'Provider Business Practice Location Address Postal Code', 'combineid',
       'combineid2_x', 'combineid3_x', 'Clinic_name_y', 'Address_y', 'City_y',
       'State_y', 'ZipCode_y', 'Phone_y', 'FAX', 'Email', 'Website', 'Name',
       'Other_info', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'combineid2_y', 'combineid3_y', 'combineid_x', 'combineid2',
       'combineid_y', 'combineid3'],
      dtype='object')

In [140]:
merged_df = merged_df[['Clinic_name_x', 'Address_x', 'City_x', 'State_x', 'ZipCode_x',
       'Phone_x', 'NPI', 'taxonomy', 'Entity Type Code',
       'Provider First Line Business Practice Location Address',
       'Provider Business Practice Location Address Postal Code', 
       'Clinic_name_y', 'Address_y', 'City_y',
       'State_y', 'ZipCode_y', 'Phone_y', 'FAX', 'Email', 'Website', 'Name',
       'Other_info', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11']]

In [143]:
merged_df = merged_df.drop_duplicates()
merged_df.shape

(132, 25)

In [144]:
merged_df.to_excel('temp.xlsx',index=False)

# Clean AOPA-NPI

In [193]:
frame4 = pd.read_excel('temp.xlsx')

In [194]:
frame4.columns

Index(['Clinic_name_x', 'Address_x', 'City_x', 'State_x', 'ZipCode_x',
       'Phone_x', 'NPI', 'taxonomy', 'Entity Type Code',
       'Provider First Line Business Practice Location Address',
       'Provider Business Practice Location Address Postal Code',
       'Clinic_name_y', 'Address_y', 'City_y', 'State_y', 'ZipCode_y',
       'Phone_y', 'FAX', 'Email', 'Website', 'Name', 'Other_info',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [195]:
npi_list = dataframe['NPI'].tolist()
merged_list = frame4['NPI'].tolist()

In [196]:
left_values = [item for item in npi_list if item not in merged_list]

In [197]:
len(left_values)

269

In [198]:
unmerged_frame = dataframe[dataframe['NPI'].isin(left_values)]

In [199]:
frame4.columns

Index(['Clinic_name_x', 'Address_x', 'City_x', 'State_x', 'ZipCode_x',
       'Phone_x', 'NPI', 'taxonomy', 'Entity Type Code',
       'Provider First Line Business Practice Location Address',
       'Provider Business Practice Location Address Postal Code',
       'Clinic_name_y', 'Address_y', 'City_y', 'State_y', 'ZipCode_y',
       'Phone_y', 'FAX', 'Email', 'Website', 'Name', 'Other_info',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [200]:
frame4 = frame4[['Clinic_name_x', 'Address_x', 'City_x', 'State_x', 'ZipCode_x',
       'Phone_x', 'Phone_y', 'FAX', 'Email', 'Website', 'Name',
       'Other_info', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'NPI']]

In [201]:
frame4.rename(columns={'Clinic_name_x': 'Clinic_name', 'Address_x': 'Address', 'City_x': 'City',
                       'State_x': 'State', 'ZipCode_x': 'ZipCode', 'Phone_x': 'Phone'}, inplace=True)

In [202]:
unmerged_frame.columns

Index(['Clinic_name', 'Address', 'City', 'State', 'ZipCode', 'Phone', 'NPI',
       'taxonomy', 'Entity Type Code',
       'Provider First Line Business Practice Location Address',
       'Provider Business Practice Location Address Postal Code', 'combineid',
       'combineid2', 'combineid3'],
      dtype='object')

In [203]:
unmerged_frame = unmerged_frame[['Clinic_name', 'Address', 'City', 'State', 'ZipCode', 'Phone', 'NPI']]

In [204]:
frame5 = pd.concat([frame4, unmerged_frame], ignore_index=True)

In [205]:
frame5.to_excel('temp3.xlsx',index=False)